In [1]:
! hdfs dfs -ls /data/wiki/en_articles_part

Found 1 items
-rwxrwxrwx   1 jovyan supergroup   76861985 2018-08-01 17:15 /data/wiki/en_articles_part/articles-part


In [55]:
%%writefile mapper.py

import sys
from line in sys.stdin:
    words = line.split()
for word in words:
    print('{0}\t{1}'.format(word,1))

Overwriting mapper.py


In [57]:
%%writefile reducer.py

import sys

curr_word = None
curr_count = 0

for line in sys.stdin:
    word,count = line.split('\t')
    count = int(count)
    if word == curr_word:
        curr_count += count
    else:
        if curr_word:
            print('{0}\t{1}'.format(curr_word,curr_count))
        curr_word = word
        curr_count = count
    if curr_word == word:
        print('{0}\t{1}'.format(curr_word,curr_count))

Overwriting reducer.py


In [4]:
%%writefile wordcount.txt

hello how are you ?
where are you ?
what are you doing for a living ?
what experience are you packing up in this lifetime ?

Writing wordcount.txt


In [5]:
!ls

mapper.py	      supervisord.log	       wordcount.txt
README.md	      supervisord.pid	       WordsRatingTask1.ipynb
reducer.py	      WordCountResolved.ipynb
StopWordsTask2.ipynb  WordCountTask0.ipynb


In [7]:
!hdfs dfs -mkdir word/


In [23]:
!hdfs dfs -chmod 777 word/

In [24]:
!hdfs dfs -ls word/

Found 1 items
-rw-r--r--   1 jovyan supergroup        124 2019-03-08 06:33 word/wordcount.txt


In [18]:
!hdfs dfs -put wordcount.txt word/

In [25]:
!hdfs dfs -ls  /

Found 3 items
drwxr-xr-x   - root   supergroup          0 2018-08-01 17:15 /data
drwxr-xr-x   - jovyan supergroup          0 2018-08-01 17:15 /datasets
drwxr-xr-x   - root   supergroup          0 2017-10-17 13:11 /user


In [26]:
!hdfs dfs -ls word/

Found 1 items
-rw-r--r--   1 jovyan supergroup        124 2019-03-08 06:33 word/wordcount.txt


In [28]:
!hdfs dfs -chmod 777 word/wordcount.txt

In [29]:
!hdfs dfs -ls word/

Found 1 items
-rwxrwxrwx   1 jovyan supergroup        124 2019-03-08 06:33 word/wordcount.txt


In [30]:
!hdfs dfs -chmod 777 word

In [31]:
!hdfs dfs -ls word

Found 1 items
-rwxrwxrwx   1 jovyan supergroup        124 2019-03-08 06:33 word/wordcount.txt


In [32]:

!hdfs dfs -mkdir outputWord

In [39]:
!hdfs dfs -chmod 777 outputWord/

In [40]:
!hdfs dfs -ls outputWord

In [42]:
%%bash

NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "mapper.py" \
    -combiner "reducer.py" \
    -reducer "reducer.py" \
    -input word \
    -output outputWord1

hdfs dfs -cat outputWord1/part-00000 | head

packageJobJar: [/tmp/hadoop-unjar5117510284629629139/] [] /tmp/streamjob513081076811918921.jar tmpDir=null


19/03/08 06:45:27 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 06:45:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 06:45:29 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/08 06:45:30 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:578)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:755)
19/03/08 06:45:30 INFO mapreduce.JobSubmitter: number of splits:2
19/03/08 06:45:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1552022483956_0001
19/03/08 06:45:31 INFO impl.YarnClientImpl: Submitted application application_1552022483956_0001
19/03/08 06:45:32 INFO mapreduce.Job: The url to tra

In [43]:
!hdfs dfs -ls

Found 3 items
drwxrwxrwx   - jovyan supergroup          0 2019-03-08 06:38 outputWord
drwxr-xr-x   - jovyan supergroup          0 2019-03-08 06:46 outputWord1
drwxrwxrwx   - jovyan supergroup          0 2019-03-08 06:33 word


In [46]:
!hdfs dfs -ls outputWord1/

In [47]:
!hdfs dfs -cat outputWord1/part-00000

cat: `outputWord1/part-00000': No such file or directory


In [48]:
!hdfs dfs -ls outputWord

In [49]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "mapper.py" \
    -combiner "reducer.py" \
    -reducer "reducer.py" \
    -input word \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

rm: `wordcount_result_1552027939648199': No such file or directory
19/03/08 06:52:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 06:52:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 06:52:32 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:578)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:755)
19/03/08 06:52:32 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/08 06:52:32 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs

In [50]:
%%bash

NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "mapper.py" \
    -combiner "reducer.py" \
    -reducer "reducer.py" \
    -input word \
    -output wordOutput



packageJobJar: [/tmp/hadoop-unjar5705795374228583009/] [] /tmp/streamjob3123988543650772852.jar tmpDir=null


-rm: Not enough arguments: expected 1 but got 0
Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-x] <path> ...]
	[-expunge]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] [-skip-empty-file] <src> <localdst>]
	[-help [cmd ...]]
	[-ls [-C] [-d] [-h] [-q] [-R] [-t] [-S] [-r] [-u

In [51]:
%%bash

NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "mapper.py" \
    -combiner "reducer.py" \
    -reducer "reducer.py" \
    -input word \
    -output wordOutputRes

packageJobJar: [/tmp/hadoop-unjar7733191181882496816/] [] /tmp/streamjob3936230658531330806.jar tmpDir=null


19/03/08 06:58:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 06:58:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 06:58:15 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/08 06:58:15 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:578)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:755)
19/03/08 06:58:15 INFO mapreduce.JobSubmitter: number of splits:2
19/03/08 06:58:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1552022483956_0004
19/03/08 06:58:16 INFO impl.YarnClientImpl: Submitted application application_1552022483956_0004
19/03/08 06:58:16 INFO mapreduce.Job: The url to tra

In [52]:
%%bash

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper.py,reducer.py \
    -mapper "mapper.py" \
    -combiner "reducer.py" \
    -reducer "reducer.py" \
    -input word \
    -output wordOutputResult

packageJobJar: [/tmp/hadoop-unjar4796742709347739943/] [] /tmp/streamjob3523755287837207169.jar tmpDir=null


19/03/08 07:01:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 07:01:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 07:01:42 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/08 07:01:42 INFO mapreduce.JobSubmitter: number of splits:2
19/03/08 07:01:42 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1552022483956_0005
19/03/08 07:01:43 INFO impl.YarnClientImpl: Submitted application application_1552022483956_0005
19/03/08 07:01:43 INFO mapreduce.Job: The url to track the job: http://23d861e5cf69:8088/proxy/application_1552022483956_0005/
19/03/08 07:01:43 INFO mapreduce.Job: Running job: job_1552022483956_0005
19/03/08 07:02:00 INFO mapreduce.Job: Job job_1552022483956_0005 running in uber mode : false
19/03/08 07:02:00 INFO mapreduce.Job:  map 0% reduce 0%
19/03/08 07:02:16 INFO mapreduce.Job: Task Id : attempt_1552022483956_0005_m_000000_0, Status : FAILED
Error: java.lang.RuntimeException:

In [58]:
%%bash

OUT_DIR="wordcount_reslt_"$(date +"%s%6N")
NUM_REDUCERS=2

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "mapper.py" \
    -combiner "reducer.py" \
    -reducer "reducer.py" \
    -input word \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

rm: `wordcount_reslt_1552033326239813': No such file or directory
19/03/08 08:22:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 08:22:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/08 08:22:17 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/08 08:22:17 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:578)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:755)
19/03/08 08:22:18 INFO mapreduce.JobSubmitter: number of splits:2
19/03/08 08:22:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1552022483956_0008
19/03/08 08:22:18 INFO impl.YarnClientImpl: Submitted application application_15520